In [ ]:
import geopandas
import pandas
import os
from pathlib import Path
import networkx


In [ ]:
allCounties = geopandas.read_file("allCounties-sorta.gpkg").set_crs(4269)
county = geopandas.GeoDataFrame()
fin = geopandas.GeoDataFrame()


ownersList = list(allCounties.owner.values)

cohort = 0

ownersList.sort()

In [ ]:
# Merges same owner parcels that are close to each other into a group.  An owner can have more than 1 group.
# This needs to be optimized. Really Really Really Really Really needs to be.  Why I didn't in the first place? PyCharm was being weird and it would be done in the morning
# --> Modify the site-joining used in the 'existingSiteAnalysis' notebook to fit the same owner constraint

# Just counts
num = 0
for owner in ownersList:

    num = num + 1
    if num % 10 == 0:
        print(num)
    combined = []
    group = allCounties[allCounties.owner == owner].to_crs("ESRI:54012").explode()

    for x in range(len(group)):
        y = x + 1
        combined.append((x,x))
        try:
            while y < len(group):
                if group.iloc[x].geometry.distance(group.iloc[y].geometry) < 100:
                    combined.append((x, y))
                y = y + 1
        except Exception as e:
            print(e)
            print("oops")


    g = networkx.Graph(combined)
    try:
        for subgraph in networkx.connected_component_subgraphs(g):
            b = list(subgraph.nodes())

            for parcs in b:
                fin = fin.append(group.iloc[parcs], ignore_index=True)
                fin.loc[fin.last_valid_index(), "gruppen"] = cohort


            cohort = cohort + 1

    except Exception as e:
        print(e)



In [ ]:
fin.to_file("finny.gpkg", driver="GPKG")

In [ ]:
procd = geopandas.read_file("finny.gpkg").set_crs(crs="ESRI:54012")


procd.loc[:, "acres"] = pandas.DataFrame(round((procd.geometry.area / 4046.86), 2)).values

In [ ]:
procd = procd.dissolve(by="gruppen")

procd = procd[procd.acres > 140]
procd = procd.drop_duplicates()

In [ ]:
substations = geopandas.read_file(str(Path("../data/Electric_Substations-shp.zip"))).to_crs("ESRI:54012")
transLines = geopandas.read_file(str(Path("../data/Electric_Power_Transmission_Lines-shp.zip"))).to_crs("ESRI:54012")

countyList = list(procd.county.drop_duplicates().values)

In [ ]:
import warnings
from geoutilities import slopeAspect
from importlib import reload
reload(slopeAspect)
procd.reset_index(inplace=True)
warnings.filterwarnings("ignore",category=RuntimeWarning)

for county in countyList:
    print(county)
    procd[procd.county == county].loc[:, "line_m"] = slopeAspect.distanceparallel(procd[(procd.county == county)], transLines, "line_m")
    procd[(procd.county == county)].loc[:, "substation_m"] = slopeAspect.distanceparallel(procd[(procd.county == county)], substations, "substation_m")

In [ ]:
procd = geopandas.to_file("procd.gpkg", driver="GPKG")

In [ ]:
for index, row in procd.iterrows():
    procd.loc[index, "owner"] = allCounties[(allCounties.orig_id == row.orig_id)].owner.values[0]

In [ ]:
dropped = procd[~((procd.substation_m > 850) & (procd.line_m > 475))].copy()


badwords = ["state of ohio", "city of", "department of natural resources", "county park", "city park", "united states of america", "metropark", "metro park", "right of way", "metro pk", "atomic energy commission", "u s a", "army"]

droplist = []

dropped["owner"] = dropped.owner.apply(lambda x : x.lower())

In [ ]:
for index, row in dropped.iterrows():
    for word in badwords:
        if word in row.owner.lower():
            droplist.append(row.gruppen)



In [ ]:
dropped = dropped[~dropped.gruppen.isin(droplist)]

import rasterstats
dropped.loc[:, "slope"] = geopandas.GeoDataFrame(rasterstats.zonal_stats(dropped.to_crs(epsg=4269), "../data/ohioSlope.tif", stats="mean"))["mean"].values
dropped.loc[:, "aspect"] = geopandas.GeoDataFrame(rasterstats.zonal_stats(dropped.to_crs(epsg=4269), "../data/ohioAspect.tif", stats="mean"))["mean"].values


dropped = dropped[dropped.slope < 11.5]

In [ ]:
dropped.to_file("reduced.gpkg", driver="GPKG")